# Homework 2

In [1]:
using Pkg; # The 'Pkg' package is used to install and load other packages
Pkg.activate("."); Pkg.instantiate(); # The AMO environment is activated and instantiated


#Pkg.status() 'gibt Status zurück

  Activating project at `c:\Users\Leonie Hergeth\Documents\GitHub\AMO_2023\LHe`


Precompiling project...

  ✗ Gurobi


  0 dependencies successfully precompiled in 5 seconds. 198 already precompiled.
  1 dependency errored. To see a full report either run `import Pkg; Pkg.precompile()` or load the package


In [2]:
using DataStructures # for SortedDict
using DataFrames
using IterTools # for product()
using Plots # for plotting, import and first plot might take a while due to compilation
using JuMP, HiGHS

At first the counter for the iteration is set and the first fixed x is set to its greatest limit. Also arrays are defined to save the values of the dual model.

To ensure that the computation is stopped after a while the limit of iterations is set to 10. For greater models it must be larger or a binary must be used which indicates whether the deviation is small enough or not. In each iteration the submodel is defined and solved. The duals are saved in the arrays and used in the master problem. If the objective value of the sub model is equal to the optimal alpha value of the master model, an optimal solution is found and the algorithm stops.

The solution is found after 4 iterations and is printed in the last line.

In [27]:
k = 1
x_fixed = 16

dual1 = [i for i in 1:10]
dual2 = [i for i in 1:10]
dual3 = [i for i in 1:10]
dual4 = [i for i in 1:10]

while k < 10 
    subModel = Model(HiGHS.Optimizer)
    @variable(subModel, y >=0) 

    @constraint(subModel,c1, -y >= -5 - x_fixed) 
    @constraint(subModel,c2, -y >= -15/2 - x_fixed/2) 
    @constraint(subModel,c3, -y >= -35/2 + x_fixed/2) 
    @constraint(subModel,c4, y >= -10 + x_fixed)

    @objective(subModel, Min, -y)

    optimize!(subModel)
    dual1[k] = dual(c1)
    dual2[k] = dual(c2)
    dual3[k] = dual(c3)
    dual4[k] = dual(c4)

    println("sub model obj value " ,objective_value(subModel))

    masterModel = Model(HiGHS.Optimizer)
    @variable(masterModel, 0 <= X <= 16) 
    @variable(masterModel, alpha)
    @objective(masterModel, Min, - 0.25 * X + alpha) 

    for kk in 1:k
        @constraint(masterModel, alpha >= dual1[kk] * (-5 - X) + dual2[kk] * (-X / 2 - 15 / 2) + dual3[kk] * (X/2 - 35/2) + dual4[kk] * (X - 10)) 
    end

    optimize!(masterModel)
    println("master obj Value " ,objective_value(masterModel))

    if abs(value(alpha) - objective_value(subModel)) < 0.0001 
        println("values are equal -> no further iteration")
        println("value of x = ", value(X), "; value of alpha = ", value(alpha), "; master obj value = " ,objective_value(masterModel), "; value of y = ", value(y), "; sub model obj Value = " ,objective_value(subModel))
        println(k, " iteration(s) needed")
        k = 10
    else
        println("value of fixed x " , x_fixed, "; value of new x ", value(X))
        x_fixed = value(X)
        k += 1
        println("iteration step end")
    end    
end

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-4); columns 0(-1); elements 0(-4) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     : -9.5000000000e+00
HiGHS run time      :          0.00
sub model obj value -9.5
Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 1 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-1); columns 0(-2); elements 0(-2) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     : -1.7500000000e+01
HiGHS run time      :          0.00
master obj Value -17.5
value of fixed x 16; value of new x 0.0
iteration step end
Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 0 cols, 0 nonzeros
0